In [1]:
import json
import lzma
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from dotenv import load_dotenv 
import numpy as np

In [2]:
# loading variables from .env file
load_dotenv("../../private_data/.env") 

# PARENT gets us to the root of the project
PARENT = "./../../"

FOLDER_TABLE = PARENT + os.getenv("FOLDER_TABLE")
FILE_FABRITIUS_DATA = PARENT + os.getenv("FILE_FABRITIUS_DATA")

FOLDER_FIGURES = PARENT + os.getenv("FOLDER_FIGURES")
IMAGES_FOLDER = PARENT + os.getenv("IMAGES_FOLDER")

DB_INPUT_ARTPIECES = PARENT + os.getenv("DB_INPUT_ARTPIECES")
DB_INPUT_ARTISTS = PARENT + os.getenv("DB_INPUT_ARTISTS")
DB_INPUT_SUBJECTMATTER = PARENT + os.getenv("DB_INPUT_SUBJECTMATTER")
##

def fixPath(path):
    return path.replace(".././", "../")

def get_image_path_from_recordID(dataset, recordID):
    """
    Given a recordID, return the local path for its image.
    """
    # Locate row in the downloaded DataFrame
    paths = dataset[
        dataset["recordID"] == recordID
    ]["low_res_filename"].values
    
    if len(paths) == 0:
        return None
    
    path = paths[0]
    # Merge: IMAGES_FOLDER + path[1:]
    merged_path = fixPath(os.path.join(IMAGES_FOLDER, path[1:]))
    return merged_path

In [3]:
DATA = pd.read_csv(FILE_FABRITIUS_DATA)
copyrightedRecordIDs = [1000, 1001, 7681]
DATA = DATA[~DATA["recordID"].isin(copyrightedRecordIDs)]
DATA

C:\Users\victo\AppData\Local\Temp\ipykernel_20812\1178667730.py:1: DtypeWarning: Columns (37,83,84,85,86,87,89,90,91,98,102,103,105,113,114,115,116,119,120,122,124,125,126,127) have mixed types. Specify dtype option on import or set low_memory=False.
  DATA = pd.read_csv(FILE_FABRITIUS_DATA)


,recordLanguage,recordID,DatabaseId,LinkToVubis,objectWork.creatorDescription,objectWork.termClassification,objectWork.workID,objectWork.titleText,objectWork.objectWorkType,objectWork.measurementsDescription,...,commissioner.commissionerName,commissioner.commissionerRole,componentsParts.componentsQuantity,componentsParts.componentsType,formalDescription.factureDescription,subjectMatter.subjectInterpretiveHistory,creator.linkAttributionDocumentation,relatedTextualReferences.citationType,materials.materialSourcePlace,creation
1,fre,10002,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,Henry de Groux,peinture (Dept. Art Moderne),12315,Portrait de Paul Verlaine,['tableau (toile)'],"[46.0, 38.0]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,fre,10004,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,Pierre Oyens,peinture (Dept. Art Moderne),3230,Les collègues,['tableau (toile)'],"[202.0, 177.0]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'earliestDate': '1884', 'latestDate': '1884'}"
3,fre,10005,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,Ferdinand Roybet,peinture (Dept. Art Moderne),6831,Portrait de femme,['tableau (panneau)'],"[81.0, 54.0]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'earliestDate': '1892', 'latestDate': '1892'}"
5,fre,10016,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,Henry de Groux,peinture (Dept. Art Moderne),12502,Christ aux outrages,['tableau (toile)'],"[173.0, 145.0]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'earliestDate': '1878', 'latestDate': '1930'}"
6,fre,1003,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,Alfred Stevens,peinture (Dept. Art Moderne),10714,Dame au chien,['tableau (panneau)'],"[26.9, 20.9]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6867,fre,9960,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,"Eugène Laermans, baron",oeuvre sur papier (Dept. Art Moderne),12311,Sur le champ,['dessin'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'earliestDate': '1895', 'latestDate': '1895'}"
6868,fre,9961,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,Fernand Khnopff,oeuvre sur papier (Dept. Art Moderne),12273,Melisande,['dessin'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'earliestDate': '1876', 'latestDate': '1921'}"
6869,fre,9962,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,Fernand Khnopff,oeuvre sur papier (Dept. Art Moderne),12264,Une aile bleue,['photographie'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'earliestDate': '1894', 'latestDate': '1894'}"
6870,fre,999,Fabritius,http://193.190.214.119/fabritiusweb/LinkToVubi...,Constant Montald,peinture (Dept. Art Moderne),10613,La baignade,['détrempe'],"[525.0, 593.0]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'earliestDate': '1913', 'latestDate': '1913'}"


In [4]:
columnsToRemove = [
    "DatabaseId",
    "LinkToVubis",
    "measurements",
    "DcaProjectNumber.DcaProjectID",
    "relatedVisualDocumentation.imageWorkRelationshipType",
    "relatedVisualDocumentation.imageType",
    "relatedVisualDocumentation.imageColor",
    "relatedVisualDocumentation.imageNumberOpacDisplay",
    "relatedVisualDocumentation.imageID",
    "relatedVisualDocumentation.imageOpacLink",
    "relatedVisualDocumentation.imageIntranetLink",
    "repositoryNumberVariant.repositoryNumberNoSpaces",
    "objectWork.objectWorkRemarks",
    "relatedWorks",
    "objectWork.titleVariant",
    "relatedWorks.relatedWorkRecordID",
    "relatedWorks.workRelationshipType",
    "relatedWorks.relatedWorkTitle",
    "relatedWorks.relatedWorkRepositoryNumber",
    "relatedWorks.relatedWorkCreator",
    "creator",
    "creator.attributionQualifier",
    "objectWork.componentsDescription",
    "relatedTextualReferences.sourceAuthID",
    "relatedTextualReferences.sourceTitle",
    "relatedTextualReferences.citationPages",
    "relatedTextualReferences.sourceAuthorName",
    "relatedTextualReferences.sourcePublisher",
    "relatedTextualReferences.sourcePublicationPlace",
    "relatedTextualReferences.sourcePublicationYear",
    "relatedTextualReferences.sourceCallNumber",
    "formalDescription.stylesPeriodsDescription",
    "context.associatedHistoricalAgent",
    "relatedTextualReferences",
    "inscriptions",
    "relatedTextualReferences.sourceParallelTitle",
    "relatedVisualDocumentation.imageViewDescription",
    "creation.creationRemarks",
    "techniques.techniquesName",
    "techniques.extentTechniques",
    "materials.extentMaterials",
    "measurements.measurementsType",
    "measurements.measurementsValue",
    "measurements.measurementsUnit",
    "measurements.measurementsShape",
    "relatedVisualDocumentation.imageOpacLinkDetails",
    "creation.placeLocAuthID",
    "creation.creationPlace",
    "context.historicalEventType",
    "context.historicalEventName",
    "context.associatedArchitecturalContext",
    "context.placeLocAuthID",
    "context.associatedHistoricalPlaceName",
    "exhibitions.exhibitionAuthID",
    "exhibitions.exhibitionTitle",
    "exhibitions.venueDates",
    "exhibitions.catalogNumber",
    "relatedWorks.linkRelatedWorksDocumentation",
    "relatedWorks.relatedWorksOpacDisplay",
    "relatedTextualReferences.sourceMaterialType",
    "creator.biographyCreator",
    "creator.linkBiographyDocumentation",
    "relatedTextualReferences.linkExcerptDocument",
    "relatedTextualReferences.excerptLinkOpacDisplay",
    "materials.materialsTechDescription",
    "copyrights.copyrightHolderName",
    "copyrights.copyrightStatement",
    "creator.attributionRemarks",
    "relatedTextualReferences.linkFurtherBibliography",
    "relatedTextualReferences.furtherBibliographyOpacDisplay",
    "commissioner.commissionerName",
    "commissioner.commissionerRole",
    "componentsParts.componentsQuantity",
    "componentsParts.componentsType",
    "formalDescription.factureDescription",
    "subjectMatter.subjectInterpretiveHistory",
    "creator.linkAttributionDocumentation",
    "relatedTextualReferences.citationType",
    "materials.materialSourcePlace",
    "creation",
    "creator.latestActivityCreator",
    "context.associatedHistoricalEventDescription", 
    "creator.specificActivityCreatorDescription",      
    "creator.earliestActivityCreator",
    "inscriptions.inscriptionAuthorship",
    "inscriptions.inscriptionLocation",
    "inscriptions.inscriptionDates",
    "inscriptions.inscriptionType",
    "materials.materialColorName",
    "materials.materialsName",
    "materials",
    "relatedTextualReferences.sourceAuthorDescription",
    "creator.copyrightStatement",
    "relatedVisualDocumentation.imageDate",
    "creator.copyrightHolderName",
    "objectWork.exhibitionDescription",
]

columns = DATA.columns
print(f"There are {len(columns)} columns in the original dataset.")
keptColumns = [column for column in columns if column not in columnsToRemove]
print(f"There are {len(keptColumns)} columns in the filtered dataset.")
DATA_ESSENTIAL = DATA[keptColumns]
# Get the % of NaN values in each column
nan_values = DATA_ESSENTIAL.isna().mean() * 100
nan_values = nan_values.sort_values(ascending=False)
nan_values

There are 129 columns in the original dataset.
There are 33 columns in the filtered dataset.


subjectMatter.generalSubjectDescription            98.005532
subjectMatter.specificSubjectIdentification        97.961858
subjectMatter.iconographicInterpretation           91.934779
subjectMatter.conceptualTerms                      89.489009
formalDescription.termStylesPeriods                87.974960
relatedVisualDocumentation.imageCopyright          80.506624
subjectMatter.iconographicTerms                    51.652351
objectWork.creationDateDescription                 51.594119
subjectMatter.subjectTerms                         38.113262
objectWork.measurementsDescription                 37.880332
creator.nationalityCreator                         31.445625
objectWork.inscriptionDescription                  25.505896
creation.earliestDate                              24.326685
creation.latestDate                                24.268452
imageColor                                         22.579706
low_res_filename                                   22.579706
imageType               

In [5]:
# Remove all rows with low_res_filename NaN values or high_res_filename NaN values
DATA_ESSENTIAL = DATA_ESSENTIAL.dropna(subset=["low_res_filename", "high_res_filename"])
# Try to open the images to check if they are corrupted
corrupted_images = []
for i, row in tqdm(DATA_ESSENTIAL.iterrows(), total=DATA_ESSENTIAL.shape[0]):
    try:
        img = plt.imread(get_image_path_from_recordID(DATA_ESSENTIAL, row["recordID"]))
    except:
        corrupted_images.append(row["recordID"])

print(corrupted_images)
print(f"There are {len(corrupted_images)} corrupted images.")
DATA_ESSENTIAL = DATA_ESSENTIAL[~DATA_ESSENTIAL["recordID"].isin(corrupted_images)]
DATA_ESSENTIAL = DATA_ESSENTIAL.reset_index(drop=True)
# Order by recordID
DATA_ESSENTIAL = DATA_ESSENTIAL.sort_values(by=["recordID"])
# Reset index
DATA_ESSENTIAL = DATA_ESSENTIAL.reset_index(drop=True)
DATA_ESSENTIAL

  0%|          | 0/5318 [00:00<?, ?it/s]

100%|██████████| 5318/5318 [00:13<00:00, 387.19it/s]

[11381, 11546, 11846, 12472, 12484, 12537, 12637, 12645, 13051, 2317, 2792, 4657, 5262, 5543, 572, 7993, 949]
There are 17 corrupted images.


,recordLanguage,recordID,objectWork.creatorDescription,objectWork.termClassification,objectWork.workID,objectWork.titleText,objectWork.objectWorkType,objectWork.measurementsDescription,objectWork.termMaterialsTech,objectWork.inscriptionDescription,...,low_res_filename,high_res_filename,creator.birthDateCreator,subjectMatter.iconographicTerms,relatedVisualDocumentation.imageCopyright,subjectMatter.conceptualTerms,subjectMatter.iconographicInterpretation,formalDescription.termStylesPeriods,subjectMatter.generalSubjectDescription,subjectMatter.specificSubjectIdentification
0,fre,64,Dirk Bouts,peinture (Dept. Art Ancien),8181,La Crucifixion,['détrempe'],"[181.5, 153.5]","['toile', 'tempera']",NaN,...,/art-foto/old/internet/Bouts-8181dig-L.jpg,/art-foto/old/intranet/Bouts-8181dig-H.jpg,1410.0,scène biblique ([SO] : Nouveau Testament : Eva...,NaN,NaN,NaN,Primitifs flamands,NaN,NaN
1,fre,77,Gustave de Smet,oeuvre sur papier (Dept. Art Moderne),11120,Jeune fille assise,['dessin'],NaN,"['papier', 'carton', 'colle', 'crayon', 'pastel']",Signature dans la bas à droite : Gust. De Smet,...,/art-foto/mod/Internet/deSmet-11120dig-L.jpg,/art-foto/mod/Intranet/deSmet-11120dig-H.jpg,1877.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,fre,78,Fernand Khnopff,peinture (Dept. Art Moderne),3980,Portrait de Mademoiselle Van der Hecht,['tableau (toile)'],"[37.0, 29.0]","['toile', ""peinture à l'huile""]",Signature et date dans le haut à droite: FERNA...,...,/art-foto/mod/Internet/Khnopff-3980dig-L.jpg,/art-foto/mod/Intranet/Khnopff-3980dig-H.jpg,1858.0,portrait (fillette : Louise van der Hecht),NaN,NaN,C'est à ses portraits que Khnopff doit sa reno...,Peinture de portraits,NaN,NaN
3,fre,79,Fernand Khnopff,peinture (Dept. Art Moderne),6366,En écoutant du Schumann,['tableau (toile)'],"[101.5, 116.5]","['toile', ""peinture à l'huile""]",Signature et date dans le haut vers la gauche ...,...,/art-foto/mod/Internet/Khnopff-6366dig-L.jpg,/art-foto/mod/Intranet/Khnopff-6366dig-H.jpg,1858.0,scène (bourgeoisie) ; méditation ; musique,NaN,Robert Schumann,La scène baigne dans une atmosphère de mélanco...,NaN,NaN,NaN
4,fre,80,Fernand Khnopff,peinture (Dept. Art Moderne),6768,Des caresses,['tableau (toile)'],"[50.5, 151.0]","['toile', ""peinture à l'huile""]",Signature et date en bas à droite : FERNAND KH...,...,/art-foto/mod/Internet/Khnopff-6768dig-L.jpg,/art-foto/mod/Intranet/Khnopff-6768dig-H.jpg,1858.0,NaN,NaN,androgyne,Sur un fond de marbre et de paysage antique de...,Symbolisme,Un léopard à tête de femme se presse contre un...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5296,fre,13322,Constantin Meunier,oeuvre sur papier (Dept. Art Moderne),11972,Page de carnet de croquis,['dessin'],NaN,"['papier', 'fusain', 'crayon']",NaN,...,/art-foto/mod/internet/Meunier-11972r-L.jpg,/art-foto/mod/intranet/Meunier-11972r-H.jpg,1831.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5297,fre,13323,George Meunier,oeuvre sur papier (Dept. Art Moderne),11973,Lot de 12 oeuvres documentaires (11 dessins + ...,['dessin'],NaN,['papier'],NaN,...,/art-foto/mod/internet/Meunier-11973_1-L.jpg,/art-foto/mod/intranet/Meunier-11973_1-H.jpg,1869.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5298,fre,13377,Eugène Devaux,oeuvre sur papier (Dept. Art Moderne),3399,Portrait de femme. Mademoiselle Collyns,['pastel'],NaN,"['papier', 'pastel', 'carton']",Signature et date vers le bas à droite : Eugèn...,...,/art-foto/mod/internet/Devaux-3399-L.jpg,/art-foto/mod/intranet/Devaux-3399-H.jpg,1822.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5299,fre,13379,George Minne,oeuvre sur papier (Dept. Art Moderne),4787,Mère et enfant,['dessin'],NaN,"['papier', 'fusain', 'carton']","Non signé, non daté",...,/art-foto/mod/internet/Minne-4787-L.jpg,/art-foto/mod/intranet/Minne-4787-H.jpg,1866.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
columnName = "formalDescription.termStylesPeriods"
# Get the non NaN values of the column
non_nan_values = DATA_ESSENTIAL[DATA_ESSENTIAL[columnName].notna()]
non_nan_values[columnName].sample(10)

4                  Symbolisme
3133                  Baroque
234                Symbolisme
2314    Peinture de portraits
2385    Peinture de portraits
2899    Peinture de portraits
2469                  Baroque
3060               Romantisme
3331                  Baroque
569               Renaissance
Name: formalDescription.termStylesPeriods, dtype: object

In [8]:
columnRenames = {
    # ArtPiece
    "recordID": "recordID", # int, PK
    "objectWork.workID": "workID", # string because of "/"
    "creator.creatorAuthID": "creatorID", # string, FK-->Artist
    "recordLanguage": "language", # string
    "objectWork.titleText": "title", # string
    "objectWork.objectWorkType": "objectWorkType", # string[]
    "objectWork.measurementsDescription": "measurements", # float[] --> Will be transformed into three columns: height, width, ratio (width/heigth)
    "objectWork.termClassification": "termClassification",  # string
    "objectWork.termMaterialsTech": "materials", # string[]
    "objectWork.inscriptionDescription": "signatureFullDescription", # string
    "objectWork.creationDateDescription": "creationFullDescription", # int
    "creation.earliestDate": "creationEarliestDate", # int
    "creation.latestDate": "creationLatestDate", # int
    "objectWork.creatorDescription": "creatorFullDescription", # Should process the ; separated values
    "formalDescription.physicalAppearanceDescription": "physicalAppearanceDescription", # string
    "imageType": "imageType", # string
    "imageColor": "imageColor", # string
    "low_res_filename": "imageLowResFilename", # string
    "high_res_filename": "imageHighResFilename", # string
    "relatedVisualDocumentation.imageCopyright": "imageCopyright", # string
    "formalDescription.termStylesPeriods": "formalDescriptionTermStylesPeriods", # string

    # Artist
    "creator.creatorAuthID": "creatorID", # string, PK
    "creator.lastNameCreator": "creatorLastName", # string
    "creator.firstNameCreator": "creatorFirstName", # string
    "creator.birthDeathDatesPlacesCreatorDescription": "creatorBirthAndDeathDescription", # string with / or - separated values
    "creator.nationalityCreator": "creatorNationality", # string
    "creator.deathDateCreator": "creatorDeathDate", # int
    "creator.birthDateCreator": "creatorBirthDate", # int

    # SubjectMatter
    "subjectMatter.subjectTerms": "subjectMatterSubjectTerms", # string
    "subjectMatter.iconographicTerms": "subjectMatterIconographicTerms", # string
    "subjectMatter.conceptualTerms": "subjectMatterConceptualTerms", # string
    "subjectMatter.iconographicInterpretation": "subjectMatterIconographicInterpretation", # string
    "subjectMatter.generalSubjectDescription": "subjectMatterGeneralSubjectDescription", # string
    "subjectMatter.specificSubjectIdentification": "subjectMatterSpecificSubjectIdentification", # string
}

# We make a new DataFrame with the renamed columns in that order
DATA_RENAMED = pd.DataFrame(columns=columnRenames.values())
for originalColumnName, newColumnName in columnRenames.items():
    DATA_RENAMED[newColumnName] = DATA_ESSENTIAL[originalColumnName]

DATA_RENAMED

,recordID,workID,creatorID,language,title,objectWorkType,measurements,termClassification,materials,signatureFullDescription,...,creatorBirthAndDeathDescription,creatorNationality,creatorDeathDate,creatorBirthDate,subjectMatterSubjectTerms,subjectMatterIconographicTerms,subjectMatterConceptualTerms,subjectMatterIconographicInterpretation,subjectMatterGeneralSubjectDescription,subjectMatterSpecificSubjectIdentification
0,64,8181,Auth:509:309,fre,La Crucifixion,['détrempe'],"[181.5, 153.5]",peinture (Dept. Art Ancien),"['toile', 'tempera']",NaN,...,Haarlem (Pays-Bas) vers 1410 ? - Louvain 1475,NaN,1475.0,1410.0,scène (homme ; femme ; enfant ; vêtement : hab...,scène biblique ([SO] : Nouveau Testament : Eva...,NaN,NaN,NaN,NaN
1,77,11120,Auth:509:255,fre,Jeune fille assise,['dessin'],NaN,oeuvre sur papier (Dept. Art Moderne),"['papier', 'carton', 'colle', 'crayon', 'pastel']",Signature dans la bas à droite : Gust. De Smet,...,Gand 1877 - Deurle / Laethem-Saint-Martin 1943,Belge,1943.0,1877.0,figure (femme ; arbre ; table ; chaise),NaN,NaN,NaN,NaN,NaN
2,78,3980,Auth:509:70,fre,Portrait de Mademoiselle Van der Hecht,['tableau (toile)'],"[37.0, 29.0]",peinture (Dept. Art Moderne),"['toile', ""peinture à l'huile""]",Signature et date dans le haut à droite: FERNA...,...,Grembergen / Termonde 1858 - Saint-Josse-ten-N...,Belge,1921.0,1858.0,figure (enfant ; de face ; robe ; noeud ; inté...,portrait (fillette : Louise van der Hecht),NaN,C'est à ses portraits que Khnopff doit sa reno...,NaN,NaN
3,79,6366,Auth:509:70,fre,En écoutant du Schumann,['tableau (toile)'],"[101.5, 116.5]",peinture (Dept. Art Moderne),"['toile', ""peinture à l'huile""]",Signature et date dans le haut vers la gauche ...,...,Grembergen / Termonde 1858 - Saint-Josse-ten-N...,Belge,1921.0,1858.0,figure (femme ; main ; intérieur ; salon ; fau...,scène (bourgeoisie) ; méditation ; musique,Robert Schumann,La scène baigne dans une atmosphère de mélanco...,NaN,NaN
4,80,6768,Auth:509:70,fre,Des caresses,['tableau (toile)'],"[50.5, 151.0]",peinture (Dept. Art Moderne),"['toile', ""peinture à l'huile""]",Signature et date en bas à droite : FERNAND KH...,...,Grembergen / Termonde 1858 - Saint-Josse-ten-N...,Belge,1921.0,1858.0,scène (être imaginaire : guépard : femme ; hom...,NaN,androgyne,Sur un fond de marbre et de paysage antique de...,Un léopard à tête de femme se presse contre un...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5296,13322,11972,Auth:509:82,fre,Page de carnet de croquis,['dessin'],NaN,oeuvre sur papier (Dept. Art Moderne),"['papier', 'fusain', 'crayon']",NaN,...,Etterbeek / Bruxelles 1831 - Ixelles / Bruxell...,Belge,1905.0,1831.0,NaN,NaN,NaN,NaN,NaN,NaN
5297,13323,11973,Auth:509:4126,fre,Lot de 12 oeuvres documentaires (11 dessins + ...,['dessin'],NaN,oeuvre sur papier (Dept. Art Moderne),['papier'],NaN,...,"Paris (France)1869 - Saint-Cloud, Hauts-de-Sei...",NaN,1942.0,1869.0,NaN,NaN,NaN,NaN,NaN,NaN
5298,13377,3399,Auth:509:1442,fre,Portrait de femme. Mademoiselle Collyns,['pastel'],NaN,oeuvre sur papier (Dept. Art Moderne),"['papier', 'pastel', 'carton']",Signature et date vers le bas à droite : Eugèn...,...,Tournai 1822 - Schaerbeek / Bruxelles 1914,Belge,1914.0,1822.0,NaN,NaN,NaN,NaN,NaN,NaN
5299,13379,4787,Auth:509:85,fre,Mère et enfant,['dessin'],NaN,oeuvre sur papier (Dept. Art Moderne),"['papier', 'fusain', 'carton']","Non signé, non daté",...,Gand 1866 - Laethem-Saint-Martin 1941,Belge,1941.0,1866.0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
constantColumns = [
    "creatorLastName",
    "creatorLastName",
    "creatorFirstName",
    "creatorBirthAndDeathDescription",
    "creatorNationality",
    "creatorBirthDate",
    "creatorDeathDate",
]
uniqueCreators = DATA_RENAMED["creatorID"].unique()
print(f"There are {len(uniqueCreators)} unique creators in the dataset.")
for creator in uniqueCreators:
    creatorArtPieces = DATA_RENAMED[DATA_RENAMED["creatorID"] == creator]
    # Check if the creator has constant values
    for constantColumn in constantColumns:
        uniqueValues = creatorArtPieces[constantColumn].unique()
        if len(uniqueValues) > 1:
            print(f"Creator {creator} has more than one value for {constantColumn}.")
            print(uniqueValues)
            print(creatorArtPieces)
            assert False

# This allows us to separate the ArtPiece and Artist data !
DATA_ARTPIECE = DATA_RENAMED.drop(columns=[
    "creatorLastName",
    "creatorLastName",
    "creatorFirstName",
    "creatorBirthAndDeathDescription",
    "creatorNationality",
    "creatorBirthDate",
    "creatorDeathDate",
    "subjectMatterSubjectTerms",
    "subjectMatterIconographicTerms",
    "subjectMatterConceptualTerms",
    "subjectMatterIconographicInterpretation",
    "subjectMatterGeneralSubjectDescription",
    "subjectMatterSpecificSubjectIdentification",
])
DATA_ARTPIECE    

There are 871 unique creators in the dataset.


,recordID,workID,creatorID,language,title,objectWorkType,measurements,termClassification,materials,signatureFullDescription,...,creationEarliestDate,creationLatestDate,creatorFullDescription,physicalAppearanceDescription,imageType,imageColor,imageLowResFilename,imageHighResFilename,imageCopyright,formalDescriptionTermStylesPeriods
0,64,8181,Auth:509:309,fre,La Crucifixion,['détrempe'],"[181.5, 153.5]",peinture (Dept. Art Ancien),"['toile', 'tempera']",NaN,...,1450.0,1460.0,Dirk Bouts,Tempera sur toile,digitale JPG,photo couleurs,/art-foto/old/internet/Bouts-8181dig-L.jpg,/art-foto/old/intranet/Bouts-8181dig-H.jpg,NaN,Primitifs flamands
1,77,11120,Auth:509:255,fre,Jeune fille assise,['dessin'],NaN,oeuvre sur papier (Dept. Art Moderne),"['papier', 'carton', 'colle', 'crayon', 'pastel']",Signature dans la bas à droite : Gust. De Smet,...,1926.0,1931.0,Gustave de Smet,Crayon et pastel sur papier collé sur carton,digitale JPG,photo couleurs,/art-foto/mod/Internet/deSmet-11120dig-L.jpg,/art-foto/mod/Intranet/deSmet-11120dig-H.jpg,NaN,NaN
2,78,3980,Auth:509:70,fre,Portrait de Mademoiselle Van der Hecht,['tableau (toile)'],"[37.0, 29.0]",peinture (Dept. Art Moderne),"['toile', ""peinture à l'huile""]",Signature et date dans le haut à droite: FERNA...,...,1883.0,1883.0,Fernand Khnopff,Huile sur toile,digitale JPG,photo couleurs,/art-foto/mod/Internet/Khnopff-3980dig-L.jpg,/art-foto/mod/Intranet/Khnopff-3980dig-H.jpg,NaN,Peinture de portraits
3,79,6366,Auth:509:70,fre,En écoutant du Schumann,['tableau (toile)'],"[101.5, 116.5]",peinture (Dept. Art Moderne),"['toile', ""peinture à l'huile""]",Signature et date dans le haut vers la gauche ...,...,1883.0,1883.0,Fernand Khnopff,Huile sur toile,digitale JPG,photo couleurs,/art-foto/mod/Internet/Khnopff-6366dig-L.jpg,/art-foto/mod/Intranet/Khnopff-6366dig-H.jpg,NaN,NaN
4,80,6768,Auth:509:70,fre,Des caresses,['tableau (toile)'],"[50.5, 151.0]",peinture (Dept. Art Moderne),"['toile', ""peinture à l'huile""]",Signature et date en bas à droite : FERNAND KH...,...,1896.0,1896.0,Fernand Khnopff,Huile sur toile,digitale JPG,photo couleurs,/art-foto/mod/Internet/Khnopff-6768dig-L.jpg,/art-foto/mod/Intranet/Khnopff-6768dig-H.jpg,NaN,Symbolisme
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5296,13322,11972,Auth:509:82,fre,Page de carnet de croquis,['dessin'],NaN,oeuvre sur papier (Dept. Art Moderne),"['papier', 'fusain', 'crayon']",NaN,...,1831.0,1905.0,Constantin Meunier,Crayon et fusain sur papier,digitale JPG,photo couleurs,/art-foto/mod/internet/Meunier-11972r-L.jpg,/art-foto/mod/intranet/Meunier-11972r-H.jpg,NaN,NaN
5297,13323,11973,Auth:509:4126,fre,Lot de 12 oeuvres documentaires (11 dessins + ...,['dessin'],NaN,oeuvre sur papier (Dept. Art Moderne),['papier'],NaN,...,1869.0,1942.0,George Meunier,Techniques diverses sur feuilles de carnet de ...,digitale JPG,photo couleurs,/art-foto/mod/internet/Meunier-11973_1-L.jpg,/art-foto/mod/intranet/Meunier-11973_1-H.jpg,NaN,NaN
5298,13377,3399,Auth:509:1442,fre,Portrait de femme. Mademoiselle Collyns,['pastel'],NaN,oeuvre sur papier (Dept. Art Moderne),"['papier', 'pastel', 'carton']",Signature et date vers le bas à droite : Eugèn...,...,1897.0,1897.0,Eugène Devaux,Pastel sur papier collé sur carton,digitale JPG,photo couleurs,/art-foto/mod/internet/Devaux-3399-L.jpg,/art-foto/mod/intranet/Devaux-3399-H.jpg,NaN,NaN
5299,13379,4787,Auth:509:85,fre,Mère et enfant,['dessin'],NaN,oeuvre sur papier (Dept. Art Moderne),"['papier', 'fusain', 'carton']","Non signé, non daté",...,1885.0,1941.0,George Minne,"Fusain sur papier, collé sur carton",digitale JPG,photo couleurs,/art-foto/mod/internet/Minne-4787-L.jpg,/art-foto/mod/intranet/Minne-4787-H.jpg,NaN,NaN


In [10]:
columnsSubjectMatter = [
    "recordID",
    "subjectMatterSubjectTerms",
    "subjectMatterIconographicTerms",
    "subjectMatterConceptualTerms",
    "subjectMatterIconographicInterpretation",
    "subjectMatterGeneralSubjectDescription",
    "subjectMatterSpecificSubjectIdentification",
]
DATA_SUBJECT_MATTER = DATA_RENAMED[columnsSubjectMatter]
DATA_SUBJECT_MATTER

,recordID,subjectMatterSubjectTerms,subjectMatterIconographicTerms,subjectMatterConceptualTerms,subjectMatterIconographicInterpretation,subjectMatterGeneralSubjectDescription,subjectMatterSpecificSubjectIdentification
0,64,scène (homme ; femme ; enfant ; vêtement : hab...,scène biblique ([SO] : Nouveau Testament : Eva...,NaN,NaN,NaN,NaN
1,77,figure (femme ; arbre ; table ; chaise),NaN,NaN,NaN,NaN,NaN
2,78,figure (enfant ; de face ; robe ; noeud ; inté...,portrait (fillette : Louise van der Hecht),NaN,C'est à ses portraits que Khnopff doit sa reno...,NaN,NaN
3,79,figure (femme ; main ; intérieur ; salon ; fau...,scène (bourgeoisie) ; méditation ; musique,Robert Schumann,La scène baigne dans une atmosphère de mélanco...,NaN,NaN
4,80,scène (être imaginaire : guépard : femme ; hom...,NaN,androgyne,Sur un fond de marbre et de paysage antique de...,Un léopard à tête de femme se presse contre un...,NaN
...,...,...,...,...,...,...,...
5296,13322,NaN,NaN,NaN,NaN,NaN,NaN
5297,13323,NaN,NaN,NaN,NaN,NaN,NaN
5298,13377,NaN,NaN,NaN,NaN,NaN,NaN
5299,13379,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
ArtPieceColumns = list(DATA_ARTPIECE.columns)
SubjectMatterColumns = list(DATA_SUBJECT_MATTER.columns)
ToRemoveColumns = set(ArtPieceColumns + SubjectMatterColumns)
# We keep "creatorID" to make the join with the Artist DataFrame
ToRemoveColumns.remove("creatorID")
DATA_ARTIST = DATA_RENAMED.drop(columns=ToRemoveColumns)
# Remove duplicates (by creatorID)
DATA_ARTIST = DATA_ARTIST.drop_duplicates(subset=["creatorID"])
DATA_ARTIST

,creatorID,creatorLastName,creatorFirstName,creatorBirthAndDeathDescription,creatorNationality,creatorDeathDate,creatorBirthDate
0,Auth:509:309,Bouts,Dirk,Haarlem (Pays-Bas) vers 1410 ? - Louvain 1475,NaN,1475.0,1410.0
1,Auth:509:255,De Smet,Gustave,Gand 1877 - Deurle / Laethem-Saint-Martin 1943,Belge,1943.0,1877.0
2,Auth:509:70,Khnopff,Fernand,Grembergen / Termonde 1858 - Saint-Josse-ten-N...,Belge,1921.0,1858.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Auth:509:1288,David,Jacques-Louis,Paris (France) 1748 - Bruxelles 1825,Français,1825.0,1748.0
...,...,...,...,...,...,...,...
5251,Auth:509:871,de Wit,Jacob,Amsterdam (Pays-Bas) 1695 - 1754,NaN,1754.0,1695.0
5260,Auth:509:664,Lombard,Lambert,Liège 1505 - 1566,NaN,1566.0,1505.0
5278,Auth:509:4039,La Fargue,Karel,Voorburg (Pays-Bas) 1738 - Den Haag (Pays-Bas)...,NaN,1793.0,1738.0
5291,Auth:509:4069,Brice,Ignace,Bruxelles 1795 - Saint-Josse-ten-Noode / Bruxe...,NaN,1866.0,1795.0


In [12]:
columnsThatAreListOfStrings = [ # Could also be a single string !!
    "objectWorkType",
    "materials",
    "formalDescriptionTermStylesPeriods"
]

def processColumnAsListOfString(df, column):
    """
        Examples:
        ['dessin'] --> 'dessin'
        ['dessin', 'peinture'] --> 'dessin|peinture'
        dessin --> 'dessin'
        'dessin' --> 'dessin'
    """
    formatted = []
    for value in df[column]:
        if pd.isna(value):
            formatted.append("")
        else:
            isList = "[" in value
            if isList:
                # ['aquarelle', 'projet']
                # 1) remove '[' and ']'
                value = value[1:-1]
                # 2) split by ', '
                values = value.split(", ")
                # 3) Remove the quotes
                values = [value[1:-1] for value in values]
                # 4) Join with '|'
                formattedValue = "|".join(values)
                formatted.append(formattedValue)
            else:
                # 'aquarelle'
                formatted.append(value)

    return formatted

for column in columnsThatAreListOfStrings:
    formattedColumn = processColumnAsListOfString(DATA_ARTPIECE, column)
    DATA_ARTPIECE[column] = formattedColumn

DATA_ARTPIECE

,recordID,workID,creatorID,language,title,objectWorkType,measurements,termClassification,materials,signatureFullDescription,...,creationEarliestDate,creationLatestDate,creatorFullDescription,physicalAppearanceDescription,imageType,imageColor,imageLowResFilename,imageHighResFilename,imageCopyright,formalDescriptionTermStylesPeriods
0,64,8181,Auth:509:309,fre,La Crucifixion,détrempe,"[181.5, 153.5]",peinture (Dept. Art Ancien),toile|tempera,NaN,...,1450.0,1460.0,Dirk Bouts,Tempera sur toile,digitale JPG,photo couleurs,/art-foto/old/internet/Bouts-8181dig-L.jpg,/art-foto/old/intranet/Bouts-8181dig-H.jpg,NaN,Primitifs flamands
1,77,11120,Auth:509:255,fre,Jeune fille assise,dessin,NaN,oeuvre sur papier (Dept. Art Moderne),papier|carton|colle|crayon|pastel,Signature dans la bas à droite : Gust. De Smet,...,1926.0,1931.0,Gustave de Smet,Crayon et pastel sur papier collé sur carton,digitale JPG,photo couleurs,/art-foto/mod/Internet/deSmet-11120dig-L.jpg,/art-foto/mod/Intranet/deSmet-11120dig-H.jpg,NaN,
2,78,3980,Auth:509:70,fre,Portrait de Mademoiselle Van der Hecht,tableau (toile),"[37.0, 29.0]",peinture (Dept. Art Moderne),toile|peinture à l'huile,Signature et date dans le haut à droite: FERNA...,...,1883.0,1883.0,Fernand Khnopff,Huile sur toile,digitale JPG,photo couleurs,/art-foto/mod/Internet/Khnopff-3980dig-L.jpg,/art-foto/mod/Intranet/Khnopff-3980dig-H.jpg,NaN,Peinture de portraits
3,79,6366,Auth:509:70,fre,En écoutant du Schumann,tableau (toile),"[101.5, 116.5]",peinture (Dept. Art Moderne),toile|peinture à l'huile,Signature et date dans le haut vers la gauche ...,...,1883.0,1883.0,Fernand Khnopff,Huile sur toile,digitale JPG,photo couleurs,/art-foto/mod/Internet/Khnopff-6366dig-L.jpg,/art-foto/mod/Intranet/Khnopff-6366dig-H.jpg,NaN,
4,80,6768,Auth:509:70,fre,Des caresses,tableau (toile),"[50.5, 151.0]",peinture (Dept. Art Moderne),toile|peinture à l'huile,Signature et date en bas à droite : FERNAND KH...,...,1896.0,1896.0,Fernand Khnopff,Huile sur toile,digitale JPG,photo couleurs,/art-foto/mod/Internet/Khnopff-6768dig-L.jpg,/art-foto/mod/Intranet/Khnopff-6768dig-H.jpg,NaN,Symbolisme
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5296,13322,11972,Auth:509:82,fre,Page de carnet de croquis,dessin,NaN,oeuvre sur papier (Dept. Art Moderne),papier|fusain|crayon,NaN,...,1831.0,1905.0,Constantin Meunier,Crayon et fusain sur papier,digitale JPG,photo couleurs,/art-foto/mod/internet/Meunier-11972r-L.jpg,/art-foto/mod/intranet/Meunier-11972r-H.jpg,NaN,
5297,13323,11973,Auth:509:4126,fre,Lot de 12 oeuvres documentaires (11 dessins + ...,dessin,NaN,oeuvre sur papier (Dept. Art Moderne),papier,NaN,...,1869.0,1942.0,George Meunier,Techniques diverses sur feuilles de carnet de ...,digitale JPG,photo couleurs,/art-foto/mod/internet/Meunier-11973_1-L.jpg,/art-foto/mod/intranet/Meunier-11973_1-H.jpg,NaN,
5298,13377,3399,Auth:509:1442,fre,Portrait de femme. Mademoiselle Collyns,pastel,NaN,oeuvre sur papier (Dept. Art Moderne),papier|pastel|carton,Signature et date vers le bas à droite : Eugèn...,...,1897.0,1897.0,Eugène Devaux,Pastel sur papier collé sur carton,digitale JPG,photo couleurs,/art-foto/mod/internet/Devaux-3399-L.jpg,/art-foto/mod/intranet/Devaux-3399-H.jpg,NaN,
5299,13379,4787,Auth:509:85,fre,Mère et enfant,dessin,NaN,oeuvre sur papier (Dept. Art Moderne),papier|fusain|carton,"Non signé, non daté",...,1885.0,1941.0,George Minne,"Fusain sur papier, collé sur carton",digitale JPG,photo couleurs,/art-foto/mod/internet/Minne-4787-L.jpg,/art-foto/mod/intranet/Minne-4787-H.jpg,NaN,


In [13]:
def processMeasurements(df):
    measurements = df["measurements"]
    heights = []
    widths = []
    ratios = []

    for measurement in measurements:
        if pd.isna(measurement):
            heights.append(np.nan)
            widths.append(np.nan)
            ratios.append(np.nan)
        else:
            isList = "[" in measurement
            if isList:
                # [1.0, 2.0]
                # 1) remove '[' and ']'
                measurement = measurement[1:-1]
                # 2) split by ', '
                values = measurement.split(", ")
                if len(values) != 2:
                    values = [values[0], -1]
                height = float(values[0])
                width = float(values[1])
                if -1 in values:
                    ratio = np.nan
                else:
                    ratio = width / height

                heights.append(height)
                widths.append(width)
                ratios.append(ratio)
            else:
                # 1.0
                height = float(measurement)
                heights.append(height)
                widths.append(np.nan)
                ratios.append(np.nan)

    df["height"] = heights
    df["width"] = widths
    df["ratio"] = ratios
    # Drop the original column
    df = df.drop(columns=["measurements"]) 
    return df

DATA_ARTPIECE = processMeasurements(DATA_ARTPIECE)
DATA_ARTPIECE

,recordID,workID,creatorID,language,title,objectWorkType,termClassification,materials,signatureFullDescription,creationFullDescription,...,physicalAppearanceDescription,imageType,imageColor,imageLowResFilename,imageHighResFilename,imageCopyright,formalDescriptionTermStylesPeriods,height,width,ratio
0,64,8181,Auth:509:309,fre,La Crucifixion,détrempe,peinture (Dept. Art Ancien),toile|tempera,NaN,1460.0,...,Tempera sur toile,digitale JPG,photo couleurs,/art-foto/old/internet/Bouts-8181dig-L.jpg,/art-foto/old/intranet/Bouts-8181dig-H.jpg,NaN,Primitifs flamands,181.5,153.5,0.845730
1,77,11120,Auth:509:255,fre,Jeune fille assise,dessin,oeuvre sur papier (Dept. Art Moderne),papier|carton|colle|crayon|pastel,Signature dans la bas à droite : Gust. De Smet,1929.0,...,Crayon et pastel sur papier collé sur carton,digitale JPG,photo couleurs,/art-foto/mod/Internet/deSmet-11120dig-L.jpg,/art-foto/mod/Intranet/deSmet-11120dig-H.jpg,NaN,,NaN,NaN,NaN
2,78,3980,Auth:509:70,fre,Portrait de Mademoiselle Van der Hecht,tableau (toile),peinture (Dept. Art Moderne),toile|peinture à l'huile,Signature et date dans le haut à droite: FERNA...,1883.0,...,Huile sur toile,digitale JPG,photo couleurs,/art-foto/mod/Internet/Khnopff-3980dig-L.jpg,/art-foto/mod/Intranet/Khnopff-3980dig-H.jpg,NaN,Peinture de portraits,37.0,29.0,0.783784
3,79,6366,Auth:509:70,fre,En écoutant du Schumann,tableau (toile),peinture (Dept. Art Moderne),toile|peinture à l'huile,Signature et date dans le haut vers la gauche ...,1883.0,...,Huile sur toile,digitale JPG,photo couleurs,/art-foto/mod/Internet/Khnopff-6366dig-L.jpg,/art-foto/mod/Intranet/Khnopff-6366dig-H.jpg,NaN,,101.5,116.5,1.147783
4,80,6768,Auth:509:70,fre,Des caresses,tableau (toile),peinture (Dept. Art Moderne),toile|peinture à l'huile,Signature et date en bas à droite : FERNAND KH...,1896.0,...,Huile sur toile,digitale JPG,photo couleurs,/art-foto/mod/Internet/Khnopff-6768dig-L.jpg,/art-foto/mod/Intranet/Khnopff-6768dig-H.jpg,NaN,Symbolisme,50.5,151.0,2.990099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5296,13322,11972,Auth:509:82,fre,Page de carnet de croquis,dessin,oeuvre sur papier (Dept. Art Moderne),papier|fusain|crayon,NaN,NaN,...,Crayon et fusain sur papier,digitale JPG,photo couleurs,/art-foto/mod/internet/Meunier-11972r-L.jpg,/art-foto/mod/intranet/Meunier-11972r-H.jpg,NaN,,NaN,NaN,NaN
5297,13323,11973,Auth:509:4126,fre,Lot de 12 oeuvres documentaires (11 dessins + ...,dessin,oeuvre sur papier (Dept. Art Moderne),papier,NaN,NaN,...,Techniques diverses sur feuilles de carnet de ...,digitale JPG,photo couleurs,/art-foto/mod/internet/Meunier-11973_1-L.jpg,/art-foto/mod/intranet/Meunier-11973_1-H.jpg,NaN,,NaN,NaN,NaN
5298,13377,3399,Auth:509:1442,fre,Portrait de femme. Mademoiselle Collyns,pastel,oeuvre sur papier (Dept. Art Moderne),papier|pastel|carton,Signature et date vers le bas à droite : Eugèn...,1897.0,...,Pastel sur papier collé sur carton,digitale JPG,photo couleurs,/art-foto/mod/internet/Devaux-3399-L.jpg,/art-foto/mod/intranet/Devaux-3399-H.jpg,NaN,,NaN,NaN,NaN
5299,13379,4787,Auth:509:85,fre,Mère et enfant,dessin,oeuvre sur papier (Dept. Art Moderne),papier|fusain|carton,"Non signé, non daté",NaN,...,"Fusain sur papier, collé sur carton",digitale JPG,photo couleurs,/art-foto/mod/internet/Minne-4787-L.jpg,/art-foto/mod/intranet/Minne-4787-H.jpg,NaN,,NaN,NaN,NaN


In [37]:
columnsThatAreIntegerArtPiece = [
    "creationFullDescription",
    "creationEarliestDate",
    "creationLatestDate",
]

def processIntegerColumns(df, columns):
    for column in columns:
        values = []
        for value in df[column]:
            try:
                values.append(int(value))
            except:
                values.append(np.nan)
        df[column] = values
    return df

DATA_ARTPIECE = processIntegerColumns(DATA_ARTPIECE, columnsThatAreIntegerArtPiece)

In [15]:
columnsThatAreIntegerArtist = [
    "creatorDeathDate",
    "creatorBirthDate",
]

DATA_ARTIST = processIntegerColumns(DATA_ARTIST, columnsThatAreIntegerArtist)
DATA_ARTIST

,creatorID,creatorLastName,creatorFirstName,creatorBirthAndDeathDescription,creatorNationality,creatorDeathDate,creatorBirthDate
0,Auth:509:309,Bouts,Dirk,Haarlem (Pays-Bas) vers 1410 ? - Louvain 1475,NaN,1475.0,1410.0
1,Auth:509:255,De Smet,Gustave,Gand 1877 - Deurle / Laethem-Saint-Martin 1943,Belge,1943.0,1877.0
2,Auth:509:70,Khnopff,Fernand,Grembergen / Termonde 1858 - Saint-Josse-ten-N...,Belge,1921.0,1858.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Auth:509:1288,David,Jacques-Louis,Paris (France) 1748 - Bruxelles 1825,Français,1825.0,1748.0
...,...,...,...,...,...,...,...
5251,Auth:509:871,de Wit,Jacob,Amsterdam (Pays-Bas) 1695 - 1754,NaN,1754.0,1695.0
5260,Auth:509:664,Lombard,Lambert,Liège 1505 - 1566,NaN,1566.0,1505.0
5278,Auth:509:4039,La Fargue,Karel,Voorburg (Pays-Bas) 1738 - Den Haag (Pays-Bas)...,NaN,1793.0,1738.0
5291,Auth:509:4069,Brice,Ignace,Bruxelles 1795 - Saint-Josse-ten-Noode / Bruxe...,NaN,1866.0,1795.0


In [16]:
DATA_SUBJECT_MATTER

,recordID,subjectMatterSubjectTerms,subjectMatterIconographicTerms,subjectMatterConceptualTerms,subjectMatterIconographicInterpretation,subjectMatterGeneralSubjectDescription,subjectMatterSpecificSubjectIdentification
0,64,scène (homme ; femme ; enfant ; vêtement : hab...,scène biblique ([SO] : Nouveau Testament : Eva...,NaN,NaN,NaN,NaN
1,77,figure (femme ; arbre ; table ; chaise),NaN,NaN,NaN,NaN,NaN
2,78,figure (enfant ; de face ; robe ; noeud ; inté...,portrait (fillette : Louise van der Hecht),NaN,C'est à ses portraits que Khnopff doit sa reno...,NaN,NaN
3,79,figure (femme ; main ; intérieur ; salon ; fau...,scène (bourgeoisie) ; méditation ; musique,Robert Schumann,La scène baigne dans une atmosphère de mélanco...,NaN,NaN
4,80,scène (être imaginaire : guépard : femme ; hom...,NaN,androgyne,Sur un fond de marbre et de paysage antique de...,Un léopard à tête de femme se presse contre un...,NaN
...,...,...,...,...,...,...,...
5296,13322,NaN,NaN,NaN,NaN,NaN,NaN
5297,13323,NaN,NaN,NaN,NaN,NaN,NaN
5298,13377,NaN,NaN,NaN,NaN,NaN,NaN
5299,13379,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Save the DataFrames
DATA_ARTPIECE.to_csv(DB_INPUT_ARTPIECES, index=False)
DATA_ARTIST.to_csv(DB_INPUT_ARTISTS, index=False)
DATA_SUBJECT_MATTER.to_csv(DB_INPUT_SUBJECTMATTER, index=False)

In [40]:
DATA_ARTPIECE["formalDescriptionTermStylesPeriods"].unique()

array(['Primitifs flamands', '', 'Peinture de portraits', 'Symbolisme',
       'Néo-classicisme', 'Romantisme', 'Orientalisme|Néo-classicisme',
       'Réalisme', 'Réalisme social', 'Peinture de genre|Romantisme',
       'Impressionnisme|Synthétisme', 'Synthétisme',
       'Néo-impressionnisme|Pointillisme|Divisionnisme',
       'Japonisme|Pointillisme', 'Préraphaélisme',
       'Impressionnisme|Réalisme', 'Pointillisme', 'Impressionnisme',
       'Primitifs flamands|Peinture de portraits', 'Renaissance',
       'Fauvisme', 'Abstraction', 'Baroque', 'Baroque-Classicisme',
       'Peinture de genre|Nature morte', 'Maniérisme', 'Caravagisme',
       'Baroque|Classicisme', 'Baroque|Maniérisme', 'Peinture de genre',
       'Classicisme', 'Contre-Réforme', 'Nature morte',
       'Classicisme|Baroque', 'Rococo', 'Peinture de paysage',
       "Peinture d'histoire|Romantisme", 'Ecole de Tervueren',
       'Peinture de genre|Peinture de paysage',
       'Peinture de genre|Peinture marine',
    